In [4]:
import numpy as np
import numpy.random as rd

In [5]:
# Input
n_MC = 10000
n = 100
n_t = 100
T_range = np.array([1, 3, 5])
mu_range = 0.01 * np.arange(1, 21)
beta_scale_factor = 10

# Model parameters
kappa = rd.uniform(0.5, 1.5, n)
X0 = c = rd.uniform(0.001, 0.051, n)
sigma_bar = rd.uniform(0, 0.2, n)
sigma = np.minimum(np.sqrt(2 * kappa * c), sigma_bar)
gamma = np.sqrt(np.square(kappa) + 2 * np.square(sigma))
beta = rd.uniform(0, 0.01, (n, n)) / beta_scale_factor
np.fill_diagonal(beta, 0)

# Choose input
T = T_range[0]
mu = mu_range[-1]
theta = np.ceil(mu*n) / T

In [6]:
def p_nocorr(t, kappa, X0, c, gamma):
    exp_gamma_t = np.exp(np.outer(t, gamma))
    p_wob = (4 * X0 * np.square(gamma) * exp_gamma_t) / \
                np.square(gamma - kappa + (gamma + kappa) * exp_gamma_t) + \
            (2 * kappa * c * (exp_gamma_t - 1)) / \
                (gamma - kappa + (gamma + kappa) * exp_gamma_t)
    return p_wob

In [40]:
n_AR = 15800
N_arr_AR = np.zeros(n_AR)

default_counts = np.zeros(n)

for i_AR in range(n_AR):
    #Step 1: initialize count and time
    t_sample = np.zeros(1)
    M = np.zeros(n)

    #Step 2: Calculate rate of transition and calculate a bounding rate
    p_wb = p_nocorr(t_sample, kappa, X0, c, gamma)
    p_bound = np.sum(p_wb)

    while t_sample[0] < T:
        #Step 3: Draw a random variable and accept/reject
        eps = rd.exponential(1 / p_bound, size=1)
        # For the CIR model with parameters given in the paper, p_wob is decreasing w.r.t time
        p_candidate_wob = np.multiply(p_nocorr(t_sample + eps, kappa, X0, c, gamma), 1 - M)
        p_candidate_wb = p_candidate_wob + np.matmul(beta, M)
        if p_bound * rd.uniform() <= np.sum(p_candidate_wb):
            #Step 4 (If there is a default): modify M
            conditional_prob = np.cumsum(p_candidate_wb) / np.sum(p_candidate_wb)
            ind = np.searchsorted(np.cumsum(p_candidate_wb) / np.sum(p_candidate_wb), rd.random())
            M[ind] = 1

        t_sample[0] = t_sample[0] + eps[0]
        
    default_counts[int(np.sum(M))] += 1

In [41]:
print(default_counts / np.sum(default_counts))

[  1.89873418e-04   8.57594937e-02   2.07151899e-01   2.57848101e-01
   2.10189873e-01   1.36075949e-01   6.34177215e-02   2.54430380e-02
   9.68354430e-03   2.84810127e-03   9.49367089e-04   4.43037975e-04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

In [43]:
default_counts[8]

153.0